In [1]:
import pandas as pd

file_path = "group/Consumer Sentiment.csv"
df = pd.read_csv(file_path)

In [2]:
df.head()

,observation_date,UMCSENT
0,1967-02-01,94.1
1,1967-03-01,NaN
2,1967-04-01,NaN
3,1967-05-01,95.9
4,1967-06-01,NaN


In [3]:
df['UMCSENT_interp'] = df['UMCSENT'].interpolate(method='linear')

df_sentiment = df['UMCSENT_interp']

In [4]:
df_wage = pd.read_excel("group/Average hourly earnings of production and nonsupervisory employees.xlsx",skiprows=10)
df_wage.head()

C:\Users\Anna Liu\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Years:,1967 to 2025,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2,1967,2.8,2.81,2.81,2.82,2.83,2.85,2.86,2.88,2.88,2.89,2.91,2.92
3,1968,2.94,2.95,2.97,2.99,3,3.01,3.03,3.04,3.07,3.08,3.1,3.11
4,1969,3.12,3.14,3.16,3.18,3.19,3.21,3.22,3.24,3.26,3.28,3.3,3.31


In [5]:
# 整理工资表：从wide format -> long format
df_wage_long = df_wage.iloc[2:].copy()  # 第3行开始是数据
df_wage_long.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 把Year列变成int
df_wage_long['Year'] = df_wage_long['Year'].astype(int)

# 把月份展开成长格式
df_wage_melted = df_wage_long.melt(id_vars=['Year'], 
                                   var_name='Month', 
                                   value_name='hourly_earning')

# 拼接成 observation_date
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
             'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_wage_melted['Month_num'] = df_wage_melted['Month'].map(month_map)
df_wage_melted['observation_date'] = pd.to_datetime(df_wage_melted['Year'].astype(str) + '-' + df_wage_melted['Month_num'] + '-01')

# 只留需要的列
df_wage_final = df_wage_melted[['observation_date', 'hourly_earning']]

# 再看看整理好的工资表
df_wage_final.head()

,observation_date,hourly_earning
0,1967-01-01,2.8
1,1968-01-01,2.94
2,1969-01-01,3.12
3,1970-01-01,3.31
4,1971-01-01,3.53


In [6]:
df['observation_date'] = pd.to_datetime(df['observation_date'])

# 合并两个表，按 observation_date 对齐
final_df = pd.merge(
    df[['observation_date', 'UMCSENT_interp']], 
    df_wage_final, 
    on='observation_date', 
    how='inner'  # 只保留两个表都有的 observation_date
)

# 查看合并结果
print(final_df.head())

  observation_date  UMCSENT_interp hourly_earning
0       1967-02-01       94.100000           2.81
1       1967-03-01       94.700000           2.81
2       1967-04-01       95.300000           2.82
3       1967-05-01       95.900000           2.83
4       1967-06-01       96.266667           2.85


In [7]:
final_df

,observation_date,UMCSENT_interp,hourly_earning
0,1967-02-01,94.100000,2.81
1,1967-03-01,94.700000,2.81
2,1967-04-01,95.300000,2.82
3,1967-05-01,95.900000,2.83
4,1967-06-01,96.266667,2.85
...,...,...,...
692,2024-10-01,70.500000,30.49
693,2024-11-01,71.800000,30.58
694,2024-12-01,74.000000,30.67
695,2025-01-01,71.700000,30.8


In [8]:
gdp_path = "group/Brave-Butters-Kelley Real Gross Domestic Product.csv"
df_gdp = pd.read_csv(gdp_path)

# 查看GDP表格前几行
df_gdp.head()

,observation_date,BBKMGDP
0,1967-01-01,6.457898
1,1967-02-01,0.985329
2,1967-03-01,0.166552
3,1967-04-01,-0.727331
4,1967-05-01,-0.534146


In [9]:
# 确保 observation_date 都是 datetime 类型
final_df['observation_date'] = pd.to_datetime(final_df['observation_date'])
df_gdp['observation_date'] = pd.to_datetime(df_gdp['observation_date'])

# 用 merge 来合并，按照 observation_date，做外连接（outer join）
merged_df = pd.merge(final_df, df_gdp, on='observation_date', how='outer')

# 查看合并后的前几行
merged_df.head()


,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP
0,1967-02-01,94.100000,2.81,0.985329
1,1967-03-01,94.700000,2.81,0.166552
2,1967-04-01,95.300000,2.82,-0.727331
3,1967-05-01,95.900000,2.83,-0.534146
4,1967-06-01,96.266667,2.85,4.148693


In [10]:
cpi_path = "group/Consumer Price Index for All Urban Consumers.xlsx"
cpi_df = pd.read_excel(cpi_path, skiprows=10)  # 通常前面有说明，跳过前10行，灵活调整

# 看一下 CPI 文件的前几行
cpi_df.head()

C:\Users\Anna Liu\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,HALF1,HALF2
1,1967,32.9,32.9,33,33.1,33.2,33.3,33.4,33.5,33.6,33.7,33.8,33.9,NaN,NaN
2,1968,34.1,34.2,34.3,34.4,34.5,34.7,34.9,35,35.1,35.3,35.4,35.5,NaN,NaN
3,1969,35.6,35.8,36.1,36.3,36.4,36.6,36.8,37,37.1,37.3,37.5,37.7,NaN,NaN
4,1970,37.8,38,38.2,38.5,38.6,38.8,39,39,39.2,39.4,39.6,39.8,NaN,NaN


In [11]:
# 整理工资表：从wide format -> long format
df_cpi = cpi_df.iloc[2:].copy()  # 第3行开始是数据
df_cpi = df_cpi.iloc[:, :13]
df_cpi.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df_cpi_melted = df_cpi.melt(id_vars=['Year'], 
                                   var_name='Month', 
                                   value_name='cpi')

# 拼接成 observation_date
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
             'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_cpi_melted['Month_num'] = df_cpi_melted['Month'].map(month_map)
df_cpi_melted['observation_date'] = pd.to_datetime(df_cpi_melted['Year'].astype(str) + '-' + df_cpi_melted['Month_num'] + '-01')

# 只留需要的列
df_cpi_final = df_cpi_melted[['observation_date', 'cpi']]

# 再看看整理好的工资表
df_cpi_final.head()

,observation_date,cpi
0,1968-01-01,34.1
1,1969-01-01,35.6
2,1970-01-01,37.8
3,1971-01-01,39.8
4,1972-01-01,41.1


In [13]:
merged_df['observation_date'] = pd.to_datetime(merged_df['observation_date'])
df_cpi_final['observation_date'] = pd.to_datetime(df_cpi_final['observation_date'])

# 按 observation_date 合并，用 outer join 保留所有日期
merged_df = pd.merge(
    merged_df,
    df_cpi_final,
    on='observation_date',
    how='inner'
)

merged_df

C:\Users\Anna Liu\AppData\Local\Temp\ipykernel_26352\1530210485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cpi_final['observation_date'] = pd.to_datetime(df_cpi_final['observation_date'])


,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,cpi_x,cpi_y
0,1968-01-01,95.766667,2.94,6.920089,34.1,34.1
1,1968-02-01,97.200000,2.95,9.107605,34.2,34.2
2,1968-03-01,95.600000,2.97,8.353572,34.3,34.3
3,1968-04-01,94.000000,2.99,5.062818,34.4,34.4
4,1968-05-01,92.400000,3,6.858492,34.5,34.5
...,...,...,...,...,...,...
681,2024-10-01,70.500000,30.49,1.544985,315.664,315.664
682,2024-11-01,71.800000,30.58,2.742999,315.493,315.493
683,2024-12-01,74.000000,30.67,3.425515,315.605,315.605
684,2025-01-01,71.700000,30.8,NaN,317.671,317.671


In [14]:
merged_df = merged_df.drop(columns=['cpi_x']).rename(columns={'cpi_y': 'CPI'})
merged_df

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI
0,1968-01-01,95.766667,2.94,6.920089,34.1
1,1968-02-01,97.200000,2.95,9.107605,34.2
2,1968-03-01,95.600000,2.97,8.353572,34.3
3,1968-04-01,94.000000,2.99,5.062818,34.4
4,1968-05-01,92.400000,3,6.858492,34.5
...,...,...,...,...,...
681,2024-10-01,70.500000,30.49,1.544985,315.664
682,2024-11-01,71.800000,30.58,2.742999,315.493
683,2024-12-01,74.000000,30.67,3.425515,315.605
684,2025-01-01,71.700000,30.8,NaN,317.671


In [15]:
core_cpi_df = pd.read_csv("group/Core CPI.csv")
core_cpi_df['observation_date'] = pd.to_datetime(core_cpi_df['observation_date'])
merged_df['observation_date'] = pd.to_datetime(merged_df['observation_date'])

# 合并，按 observation_date，outer join
merged_df = pd.merge(
    merged_df,
    core_cpi_df,
    on='observation_date',
    how='inner'
)
merged_df = merged_df.iloc[:-12, :]
merged_df

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL
0,1968-01-01,95.766667,2.94,6.920089,34.1,35.500
1,1968-02-01,97.200000,2.95,9.107605,34.2,35.700
2,1968-03-01,95.600000,2.97,8.353572,34.3,35.800
3,1968-04-01,94.000000,2.99,5.062818,34.4,35.900
4,1968-05-01,92.400000,3,6.858492,34.5,36.000
...,...,...,...,...,...,...
669,2023-10-01,63.800000,29.27,2.307123,307.671,311.433
670,2023-11-01,61.300000,29.38,3.047752,307.051,312.373
671,2023-12-01,69.700000,29.48,1.578014,306.746,313.230
672,2024-01-01,79.000000,29.61,-0.497279,308.417,314.389


In [16]:
path = "group/Discouraged Workers.xlsx"
discouraged_df = pd.read_excel(path, skiprows=12)  # 通常前面有说明，跳过前10行，灵活调整

# 看一下 CPI 文件的前几行
discouraged_df.head()

C:\Users\Anna Liu\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
1,1994,571,479,518,499,465,533,511,498,550,473,455,442
2,1995,419,429,441,382,424,365,430,417,360,423,408,422
3,1996,389,444,439,399,375,414,399,423,413,385,352,332
4,1997,378,356,346,376,359,353,293,317,346,311,337,342


In [17]:
# 整理工资表：从wide format -> long format
df_discouraged = discouraged_df.iloc[2:].copy()  # 第3行开始是数据
df_discouraged.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 把Year列变成int
# df_discouraged['Year'] = df_discouraged['Year'].astype(int)

# 把月份展开成长格式
df_discouraged = df_discouraged.melt(id_vars=['Year'], 
                                   var_name='Month', 
                                   value_name='discouraged')

# 拼接成 observation_date
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
             'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_discouraged['Month_num'] = df_discouraged['Month'].map(month_map)
df_discouraged['observation_date'] = pd.to_datetime(df_discouraged['Year'].astype(str) + '-' + df_discouraged['Month_num'] + '-01')

# 只留需要的列
df_discouraged = df_discouraged[['observation_date', 'discouraged']]

# 再看看整理好的工资表
df_discouraged.head()

,observation_date,discouraged
0,1995-01-01,419
1,1996-01-01,389
2,1997-01-01,378
3,1998-01-01,356
4,1999-01-01,323


In [18]:
merged_df['observation_date'] = pd.to_datetime(merged_df['observation_date'])
df_discouraged['observation_date'] = pd.to_datetime(df_discouraged['observation_date'])

# 按 observation_date 合并，用 outer join 保留所有日期
merged_df7 = pd.merge(
    merged_df,
    df_discouraged,
    on='observation_date',
    how='outer'
)

merged_df7

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged
0,1968-01-01,95.766667,2.94,6.920089,34.1,35.5,NaN
1,1968-02-01,97.200000,2.95,9.107605,34.2,35.7,NaN
2,1968-03-01,95.600000,2.97,8.353572,34.3,35.8,NaN
3,1968-04-01,94.000000,2.99,5.062818,34.4,35.9,NaN
4,1968-05-01,92.400000,3,6.858492,34.5,36.0,NaN
...,...,...,...,...,...,...,...
691,2025-10-01,NaN,NaN,NaN,NaN,NaN,NaN
692,2024-11-01,NaN,NaN,NaN,NaN,NaN,401
693,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN
694,2024-12-01,NaN,NaN,NaN,NaN,NaN,480


In [19]:
fedfunds_df = pd.read_csv("group/Federal Funds Effective Rate.csv")

# 确保 observation_date 是 datetime 类型
fedfunds_df['observation_date'] = pd.to_datetime(fedfunds_df['observation_date'])
merged_df7['observation_date'] = pd.to_datetime(merged_df7['observation_date'])

# 合并
merged_df8 = pd.merge(
    merged_df7,
    fedfunds_df,
    on='observation_date',
    how='outer'
)

merged_df8

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS
0,1968-01-01,95.766667,2.94,6.920089,34.1,35.5,NaN,4.61
1,1968-02-01,97.200000,2.95,9.107605,34.2,35.7,NaN,4.71
2,1968-03-01,95.600000,2.97,8.353572,34.3,35.8,NaN,5.05
3,1968-04-01,94.000000,2.99,5.062818,34.4,35.9,NaN,5.76
4,1968-05-01,92.400000,3,6.858492,34.5,36.0,NaN,6.12
...,...,...,...,...,...,...,...,...
703,1967-08-01,NaN,NaN,NaN,NaN,NaN,NaN,3.90
704,1967-09-01,NaN,NaN,NaN,NaN,NaN,NaN,3.99
705,1967-10-01,NaN,NaN,NaN,NaN,NaN,NaN,3.88
706,1967-11-01,NaN,NaN,NaN,NaN,NaN,NaN,4.13


In [23]:
job_losers_df = pd.read_excel("group/Job Losers.xlsx", skiprows=12)
# 整理工资表：从wide format -> long format
df_job_losers = job_losers_df.iloc[2:].copy()  # 第3行开始是数据
df_job_losers.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


# 把月份展开成长格式
df_job_losers = df_job_losers.melt(id_vars=['Year'], 
                                   var_name='Month', 
                                   value_name='num_losers')

# 拼接成 observation_date
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
             'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_job_losers['Month_num'] = df_job_losers['Month'].map(month_map)
df_job_losers['observation_date'] = pd.to_datetime(df_job_losers['Year'].astype(str) + '-' + df_job_losers['Month_num'] + '-01')

# 只留需要的列
df_job_losers = df_job_losers[['observation_date', 'num_losers']]

# 再看看整理好的工资表
df_job_losers.head()

C:\Users\Anna Liu\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,observation_date,num_losers
0,1969-01-01,979.0
1,1970-01-01,1245.0
2,1971-01-01,2331.0
3,1972-01-01,2238.0
4,1973-01-01,1791.0


In [25]:
merged_df8['observation_date'] = pd.to_datetime(merged_df8['observation_date'])
df_job_losers['observation_date'] = pd.to_datetime(df_job_losers['observation_date'])

# 按 observation_date 合并，用 outer join 保留所有日期
merged_df9 = pd.merge(
    merged_df8,
    df_job_losers,
    on='observation_date',
    how='inner'
)

merged_df9

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers
0,1969-01-01,96.033333,3.12,8.707461,35.6,37.3,NaN,6.30,979.0
1,1969-02-01,98.200000,3.14,6.724535,35.8,37.6,NaN,6.61,974.0
2,1969-03-01,95.966667,3.16,3.036386,36.1,37.8,NaN,6.79,999.0
3,1969-04-01,93.733333,3.18,-0.949859,36.3,38.1,NaN,7.41,1044.0
4,1969-05-01,91.500000,3.19,-1.008622,36.4,38.1,NaN,8.67,995.0
...,...,...,...,...,...,...,...,...,...
679,2025-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,2024-11-01,NaN,NaN,NaN,NaN,NaN,401,4.64,3394.0
681,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
682,2024-12-01,NaN,NaN,NaN,NaN,NaN,480,4.48,3251.0


In [29]:
# 读取 monthly continued claims 数据
claims_df = pd.read_csv("group/monthly_continued_claims.csv")

claims_df = claims_df.rename(columns={'month':'observation_date'})
# 确保 observation_date 是 datetime 类型
claims_df['observation_date'] = pd.to_datetime(claims_df['observation_date'])
merged_df9['observation_date'] = pd.to_datetime(merged_df9['observation_date'])

# 合并
merged_df10 = pd.merge(
    merged_df9,
    claims_df,
    on='observation_date',
    how='outer'
)

merged_df10

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims
0,1969-01-01,96.033333,3.12,8.707461,35.6,37.3,NaN,6.30,979.0,3.391645e+06
1,1969-02-01,98.200000,3.14,6.724535,35.8,37.6,NaN,6.61,974.0,3.569714e+06
2,1969-03-01,95.966667,3.16,3.036386,36.1,37.8,NaN,6.79,999.0,4.677065e+06
3,1969-04-01,93.733333,3.18,-0.949859,36.3,38.1,NaN,7.41,1044.0,3.239033e+06
4,1969-05-01,91.500000,3.19,-1.008622,36.4,38.1,NaN,8.67,995.0,4.148419e+06
...,...,...,...,...,...,...,...,...,...,...
703,1968-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.491323e+06
704,1968-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.382667e+06
705,1968-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.247710e+06
706,1968-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.354333e+06


In [36]:
# 读取 PCE 数据
pce_df = pd.read_csv("group/Personal Consumption Expenditures Chain-type Price Index.csv")

# 确保 observation_date 是 datetime 类型
pce_df['observation_date'] = pd.to_datetime(pce_df['observation_date'])
merged_df10['observation_date'] = pd.to_datetime(merged_df10['observation_date'])

# 合并
merged_df11 = pd.merge(
    merged_df10,
    pce_df,
    on='observation_date',
    how='outer'
)

merged_df11

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI
0,1969-01-01,96.033333,3.12,8.707461,35.6,37.3,NaN,6.30,979.0,3.391645e+06,18.476
1,1969-02-01,98.200000,3.14,6.724535,35.8,37.6,NaN,6.61,974.0,3.569714e+06,18.523
2,1969-03-01,95.966667,3.16,3.036386,36.1,37.8,NaN,6.79,999.0,4.677065e+06,18.610
3,1969-04-01,93.733333,3.18,-0.949859,36.3,38.1,NaN,7.41,1044.0,3.239033e+06,18.692
4,1969-05-01,91.500000,3.19,-1.008622,36.4,38.1,NaN,8.67,995.0,4.148419e+06,18.767
...,...,...,...,...,...,...,...,...,...,...,...
703,1968-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.491323e+06,18.161
704,1968-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.382667e+06,18.224
705,1968-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.247710e+06,18.304
706,1968-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.354333e+06,18.366


In [37]:
# 读取 Personal Consumption Expenditures 数据
pce_df2 = pd.read_csv("group/Personal Consumption Expenditures.csv")

# 确保 observation_date 是 datetime 类型
pce_df2['observation_date'] = pd.to_datetime(pce_df2['observation_date'])
merged_df11['observation_date'] = pd.to_datetime(merged_df11['observation_date'])

# 合并
merged_df12 = pd.merge(
    merged_df11,
    pce_df2,
    on='observation_date',
    how='outer'
)

# 查看合并后的结果
merged_df12.head()

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE
0,1969-01-01,96.033333,3.12,8.707461,35.6,37.3,NaN,6.30,979.0,3.391645e+06,18.476,583.5
1,1969-02-01,98.200000,3.14,6.724535,35.8,37.6,NaN,6.61,974.0,3.569714e+06,18.523,588.7
2,1969-03-01,95.966667,3.16,3.036386,36.1,37.8,NaN,6.79,999.0,4.677065e+06,18.610,588.9
3,1969-04-01,93.733333,3.18,-0.949859,36.3,38.1,NaN,7.41,1044.0,3.239033e+06,18.692,593.9
4,1969-05-01,91.500000,3.19,-1.008622,36.4,38.1,NaN,8.67,995.0,4.148419e+06,18.767,600.3


In [40]:
unemployment_df = pd.read_excel("group/Unemployment Level.xlsx", skiprows=12)
# 整理工资表：从wide format -> long format
df_unemployment = unemployment_df.iloc[2:].copy()  # 第3行开始是数据
df_unemployment.columns = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


# 把月份展开成长格式
df_unemployment = df_unemployment.melt(id_vars=['Year'], 
                                   var_name='Month', 
                                   value_name='unemployment_level')

# 拼接成 observation_date
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
             'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_unemployment['Month_num'] = df_unemployment['Month'].map(month_map)
df_unemployment['observation_date'] = pd.to_datetime(df_unemployment['Year'].astype(str) + '-' + df_unemployment['Month_num'] + '-01')

# 只留需要的列
df_unemployment = df_unemployment[['observation_date', 'unemployment_level']]

# 再看看整理好的工资表
df_unemployment.head()

C:\Users\Anna Liu\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,observation_date,unemployment_level
0,1970-01-01,3201.0
1,1971-01-01,4986.0
2,1972-01-01,5019.0
3,1973-01-01,4326.0
4,1974-01-01,4644.0


In [42]:
merged_df12['observation_date'] = pd.to_datetime(merged_df12['observation_date'])
df_unemployment['observation_date'] = pd.to_datetime(df_unemployment['observation_date'])

# 按 observation_date 合并，用 outer join 保留所有日期
merged_df13 = pd.merge(
    merged_df12,
    df_unemployment,
    on='observation_date',
    how='inner'
)

merged_df13

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE,unemployment_level
0,1970-01-01,78.633333,3.31,-1.014119,37.8,39.6,NaN,8.98,1245.0,5.668871e+06,19.377,628.7,3201.0
1,1970-02-01,78.100000,3.33,1.906225,38,39.8,NaN,8.98,1403.0,5.684000e+06,19.454,634.0,3453.0
2,1970-03-01,77.200000,3.36,-1.392509,38.2,40.1,NaN,7.76,1511.0,6.136968e+06,19.502,632.3,3635.0
3,1970-04-01,76.300000,3.36,-1.604366,38.5,40.4,NaN,8.10,1602.0,5.265033e+06,19.601,636.0,3797.0
4,1970-05-01,75.400000,3.37,2.520264,38.6,40.5,NaN,7.95,1829.0,7.964774e+06,19.663,642.4,3919.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2025-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,2024-11-01,NaN,NaN,NaN,NaN,NaN,401,4.64,3394.0,7.840000e+06,124.399,20235.1,7121.0
669,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,2024-12-01,NaN,NaN,NaN,NaN,NaN,480,4.48,3251.0,7.681710e+06,124.769,20408.1,6886.0


In [44]:
merged_df13

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE,unemployment_level
0,1970-01-01,78.633333,3.31,-1.014119,37.8,39.6,NaN,8.98,1245.0,5.668871e+06,19.377,628.7,3201.0
1,1970-02-01,78.100000,3.33,1.906225,38,39.8,NaN,8.98,1403.0,5.684000e+06,19.454,634.0,3453.0
2,1970-03-01,77.200000,3.36,-1.392509,38.2,40.1,NaN,7.76,1511.0,6.136968e+06,19.502,632.3,3635.0
3,1970-04-01,76.300000,3.36,-1.604366,38.5,40.4,NaN,8.10,1602.0,5.265033e+06,19.601,636.0,3797.0
4,1970-05-01,75.400000,3.37,2.520264,38.6,40.5,NaN,7.95,1829.0,7.964774e+06,19.663,642.4,3919.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2025-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,2024-11-01,NaN,NaN,NaN,NaN,NaN,401,4.64,3394.0,7.840000e+06,124.399,20235.1,7121.0
669,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,2024-12-01,NaN,NaN,NaN,NaN,NaN,480,4.48,3251.0,7.681710e+06,124.769,20408.1,6886.0


In [45]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 然后
merged_df13  # 或者直接 df

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE,unemployment_level
0,1970-01-01,78.633333,3.31,-1.014119,37.8,39.600,NaN,8.98,1245.0,5.668871e+06,19.377,628.7,3201.0
1,1970-02-01,78.100000,3.33,1.906225,38,39.800,NaN,8.98,1403.0,5.684000e+06,19.454,634.0,3453.0
2,1970-03-01,77.200000,3.36,-1.392509,38.2,40.100,NaN,7.76,1511.0,6.136968e+06,19.502,632.3,3635.0
3,1970-04-01,76.300000,3.36,-1.604366,38.5,40.400,NaN,8.10,1602.0,5.265033e+06,19.601,636.0,3797.0
4,1970-05-01,75.400000,3.37,2.520264,38.6,40.500,NaN,7.95,1829.0,7.964774e+06,19.663,642.4,3919.0
5,1970-06-01,76.133333,3.4,5.741429,38.8,40.800,NaN,7.61,1805.0,5.889633e+06,19.714,646.3,4071.0
6,1970-07-01,76.866667,3.41,7.467100,39,40.900,NaN,7.21,1859.0,5.903613e+06,19.786,648.5,4175.0
7,1970-08-01,77.600000,3.44,1.222761,39,41.100,NaN,6.62,1944.0,8.487677e+06,19.838,652.9,4256.0
8,1970-09-01,75.866667,3.45,-5.837982,39.2,41.300,NaN,6.29,2078.0,6.578467e+06,19.927,659.1,4456.0
9,1970-10-01,74.133333,3.45,-11.549699,39.4,41.500,NaN,6.20,2233.0,9.186226e+06,20.026,658.3,4591.0


In [46]:
merged_df13 = merged_df13.sort_values(by='observation_date').reset_index(drop=True)
merged_df13 

,observation_date,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE,unemployment_level
0,1970-01-01,78.633333,3.31,-1.014119,37.8,39.600,NaN,8.98,1245.0,5.668871e+06,19.377,628.7,3201.0
1,1970-02-01,78.100000,3.33,1.906225,38,39.800,NaN,8.98,1403.0,5.684000e+06,19.454,634.0,3453.0
2,1970-03-01,77.200000,3.36,-1.392509,38.2,40.100,NaN,7.76,1511.0,6.136968e+06,19.502,632.3,3635.0
3,1970-04-01,76.300000,3.36,-1.604366,38.5,40.400,NaN,8.10,1602.0,5.265033e+06,19.601,636.0,3797.0
4,1970-05-01,75.400000,3.37,2.520264,38.6,40.500,NaN,7.95,1829.0,7.964774e+06,19.663,642.4,3919.0
5,1970-06-01,76.133333,3.4,5.741429,38.8,40.800,NaN,7.61,1805.0,5.889633e+06,19.714,646.3,4071.0
6,1970-07-01,76.866667,3.41,7.467100,39,40.900,NaN,7.21,1859.0,5.903613e+06,19.786,648.5,4175.0
7,1970-08-01,77.600000,3.44,1.222761,39,41.100,NaN,6.62,1944.0,8.487677e+06,19.838,652.9,4256.0
8,1970-09-01,75.866667,3.45,-5.837982,39.2,41.300,NaN,6.29,2078.0,6.578467e+06,19.927,659.1,4456.0
9,1970-10-01,74.133333,3.45,-11.549699,39.4,41.500,NaN,6.20,2233.0,9.186226e+06,20.026,658.3,4591.0


In [53]:


merged_df13['discouraged'] = merged_df13['discouraged'].interpolate(method='linear',limit_direction='both')


merged_df13

C:\Users\Anna Liu\AppData\Local\Temp\ipykernel_26352\3399263274.py:1: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merged_df13['discouraged'] = merged_df13['discouraged'].interpolate(method='linear',limit_direction='both')


,UMCSENT_interp,hourly_earning,BBKMGDP,CPI,CPILFESL,discouraged,FEDFUNDS,num_losers,continued_claims,PCEPI,PCE,unemployment_level
observation_date,,,,,,,,,,,,
1970-01-01,78.633333,3.31,-1.014119,37.8,39.600,NaN,8.98,1245.0,5.668871e+06,19.377,628.7,3201.0
1970-02-01,78.100000,3.33,1.906225,38,39.800,NaN,8.98,1403.0,5.684000e+06,19.454,634.0,3453.0
1970-03-01,77.200000,3.36,-1.392509,38.2,40.100,NaN,7.76,1511.0,6.136968e+06,19.502,632.3,3635.0
1970-04-01,76.300000,3.36,-1.604366,38.5,40.400,NaN,8.10,1602.0,5.265033e+06,19.601,636.0,3797.0
1970-05-01,75.400000,3.37,2.520264,38.6,40.500,NaN,7.95,1829.0,7.964774e+06,19.663,642.4,3919.0
1970-06-01,76.133333,3.4,5.741429,38.8,40.800,NaN,7.61,1805.0,5.889633e+06,19.714,646.3,4071.0
1970-07-01,76.866667,3.41,7.467100,39,40.900,NaN,7.21,1859.0,5.903613e+06,19.786,648.5,4175.0
1970-08-01,77.600000,3.44,1.222761,39,41.100,NaN,6.62,1944.0,8.487677e+06,19.838,652.9,4256.0
1970-09-01,75.866667,3.45,-5.837982,39.2,41.300,NaN,6.29,2078.0,6.578467e+06,19.927,659.1,4456.0


In [56]:
merged_df13.to_csv('project_data.csv', index=True)